In [1]:
import numpy as np

from numpy.testing import assert_allclose

In [23]:
def householder(vec):
    vec = np.asarray(vec, dtype=float)
    if len(vec) <= 1:
        return vec, np.eye(len(vec))
    #просто последовательно реализуем алгоритм
    n = len(vec)
    outvec = np.zeros(n)
    outvec[0] = np.linalg.norm(vec)
    u = (vec - outvec) / np.linalg.norm(vec - outvec)
    H = np.eye(n) - 2 * u.reshape(n,1) @ u.reshape(1,n)
    return outvec, H

In [24]:
v = np.array([1, 2, 3])
v1, h = householder(v)
assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1, atol = 1e-10)


In [25]:
rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

In [36]:
def qr_decomp(a):

    A = np.array(a, copy=True, dtype=float)
    m, n = A.shape
    
    H = np.eye(m) #матрица, которая будет "запоминать" все преобразования
    Ai, h = householder(A[:,0]) #Ai никак не будет использоваться, нужен только потому что householder возвращает два значения
    A = h @ A
    H = h @ H
    for i in range(1,n):
        Ai, h = householder(A[i:,i]) #ищем Хаусхолдер для среза из поддиагональных и диагональных элементов для каждого столбца
        
        #собираем матрицу из Хаусхолдера
        I = np.eye(i)              #единичная верхний левый угол
        hr = np.zeros((i,m-i))     #матрица нулей верхний правый угол
        bl = np.zeros((m-i,i))     #матрица нулей нжний правый угол
        top = np.hstack((I,hr))    #верхняя часть
        bot = np.hstack((bl,h))    #нижняя
        
        E = np.vstack((top,bot))   #наша матрица накенец получена!!!
        H = H @ E
        A = E @ A
  
    return H, A

In [37]:
np.set_printoptions(suppress=True)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

assert_allclose(np.dot(q, r), a)

In [38]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

In [39]:
print(r)
print("----")
print(rr)
print("-------------------------------")
print(q)
print("----")
print(qq)

[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.         -0.          0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.         -0.         -0.        ]]
----
[[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
-------------------------------
[[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]]
----
[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907

мы использовали Хаусхолдер, который выдавал модули векторов, поэтому у нас везде положительные значения, во втроенном в итоговых значениях r появлялись отрицательные

Part III. Avoid forming Householder matrices explicitly.

In [41]:
def ur_decomp(a):
    
    def by_u(vec):   #функция поиска вектора u
        n = len(vec)
        outvec = np.zeros(n)
        outvec[0] = np.linalg.norm(vec)
        u = (vec - outvec) / np.linalg.norm(vec - outvec)
        return u
       
    A = np.array(a, copy=True, dtype=float)
    m, n = np.shape(A)
    U = []  #здесь будем хранить все u
    U.append(by_u(A[0:,0]))
    v = np.array(U[0])
    A = A - 2*v.reshape(len(v),1)@(v.reshape(1,len(v))@A)    #считаем на следующем шаге матрицу A
    stop = min(n,m)
    for i in range(1,stop):
        U.append(np.hstack((np.zeros(i),by_u(A[i:,i]))))    #добавляем веткора u, сразу удлинняя их
        v = np.array(U[i])
        A = A - 2*v.reshape(len(v),1)@(v.reshape(1,len(v))@A)
    return U, A

In [82]:
def build_from_u(u, a):
    n = len(u)       #число столбцов (количество u)
    m = len(u[0])    #число строк (длина u)
    for i in range(n-1,-1,-1):
        a = a - 2 * u[i].reshape(m,1)@(u[i].reshape(1,m)@a)
    return a

In [83]:
u, r = ur_decomp(a)

In [85]:
print(build_from_u(u,r)-a)

[[ 0.  0.  0.]
 [-0.  0.  0.]
 [ 0. -0. -0.]
 [-0.  0. -0.]
 [ 0. -0. -0.]]


In [86]:
monster = np.zeros((59,70))
for i in range(len(monster)):
    for j in range(len(monster[0])):
        monster[i][j] = i + i*(j+2) +j**2
print(monster)

[[   0.    1.    4. ... 4489. 4624. 4761.]
 [   3.    5.    9. ... 4559. 4695. 4833.]
 [   6.    9.   14. ... 4629. 4766. 4905.]
 ...
 [ 168.  225.  284. ... 8409. 8600. 8793.]
 [ 171.  229.  289. ... 8479. 8671. 8865.]
 [ 174.  233.  294. ... 8549. 8742. 8937.]]


In [94]:
mu, mr = ur_decomp(monster)
print(build_from_u(mu,mr)-monster)

[[ 0.  0. -0. ...  0.  0. -0.]
 [-0. -0. -0. ... -0. -0. -0.]
 [-0. -0.  0. ...  0.  0.  0.]
 ...
 [-0. -0.  0. ... -0. -0.  0.]
 [-0. -0.  0. ... -0. -0.  0.]
 [-0. -0.  0. ... -0. -0.  0.]]
